In [ ]:
import os
import numpy as np


class MT:

    # The coefficients for MT19937 are:
    (w, N, M, r) = (32, 624, 397, 31)
    MATRIX_A = 0x9908b0df
    (u, d) = (11, 0xffffffff)
    (s, b) = (7, 0x9d2c5680)
    (t, c) = (15, 0xefc60000)
    l = 18
    f = 1812433253
    (UPPER_MASK, LOWER_MASK) = (0x80000000, 0x7fffffff)

    def __init__(self, seed=None):
        if seed is None:
            seed = int.from_bytes(os.urandom(4), 'big')

        self.mt = np.zeros(MT.N, dtype=np.uint32)
        self.mt[0] = seed

        for i in range(1, MT.N):
            mt_value = MT.f * (self.mt[i-1] ^ (self.mt[i-1] >> (MT.w - 2))) + i
            self.mt[i] = mt_value

        self.index = MT.N

    def _twist(self):
        for i in range(MT.N):
            y = (self.mt[i] & MT.UPPER_MASK) | (self.mt[(i + 1) % MT.N] & MT.LOWER_MASK)
            self.mt[i] = self.mt[(i + MT.M) % MT.N] ^ (y >> 1)
            if y % 2 != 0:
                self.mt[i] ^= MT.MATRIX_A

    def random_int(self):
        # twisting
        if self.index >= self.N:
            self._twist()
            self.index = 0

        # extracting
        y = self.mt[self.index]

        # tempering
        y ^= (y >> MT.u) & MT.d
        y ^= (y << MT.s) & MT.b
        y ^= (y << MT.t) & MT.c
        y ^= (y >> MT.l)
        self.index += 1

        return y

In [ ]:
mt = MT(12345)
for i in range(10):
    print(mt.random_int())

3992670690
3823185381
1358822685
561383553
789925284
170765737
878579710
3549516158
2438360421
2285257250


In [ ]:
import numpy as np


mt = MT()

num_samples = 20
sample_size = 1000
samples = []

for i in range(num_samples):
    samples.append([mt.random_int() for _ in range(sample_size)])

# means = [np.mean(sample) for sample in samples]
# stds = [np.std(sample) for sample in samples]
# cvs = [(std/mean) * 100 for std, mean in zip(stds, means)]

for i, sample in enumerate(samples):
    mean = np.mean(sample)
    std = np.std(sample)
    cv = (std / mean) * 100
    print(f"{i}\t{mean = }\tstd = {std:.4f}\tcv = {cv:.4f}%")

0	mean = 2111927429.596	std = 1242074139.7308	cv = 58.8123%
1	mean = 2181751172.787	std = 1240319515.6084	cv = 56.8497%
2	mean = 2071223834.493	std = 1247246311.0180	cv = 60.2178%
3	mean = 2076005728.895	std = 1232491318.6458	cv = 59.3684%
4	mean = 2210631321.212	std = 1233836881.9926	cv = 55.8138%
5	mean = 2130278344.15	std = 1245855177.1205	cv = 58.4832%
6	mean = 2120534155.503	std = 1219090360.1798	cv = 57.4898%
7	mean = 2113656260.769	std = 1258430193.0591	cv = 59.5381%
8	mean = 2122643572.214	std = 1236255741.1831	cv = 58.2413%
9	mean = 2070549408.452	std = 1228472842.8669	cv = 59.3308%
10	mean = 2093174074.567	std = 1242217680.9364	cv = 59.3461%
11	mean = 2168113498.561	std = 1220868384.3413	cv = 56.3102%
12	mean = 2129553842.841	std = 1241189282.9612	cv = 58.2840%
13	mean = 2161409782.938	std = 1244001502.1300	cv = 57.5551%
14	mean = 2089118543.467	std = 1265924417.9155	cv = 60.5961%
15	mean = 2243130037.53	std = 1240719150.4373	cv = 55.3120%
16	mean = 2161250161.658	std = 12392

In [ ]:
import numpy as np
from scipy.stats import chi2


def chi_squared_test(sample, num_bins=10):
    min_value = min(sample)
    max_value = max(sample)

    bin_width = (max_value - min_value) / num_bins

    observed, _ = np.histogram(sample, bins=num_bins, range=(min_value, max_value))
    expected = len(sample) / num_bins

    chi_squared = np.sum(((observed - expected)**2) / expected)

    p_value = 1 - chi2.cdf(chi_squared, df=num_bins-1)

    return chi_squared, p_value


for sample in samples:
    chi_squared, p_value = chi_squared_test(sample)
    if p_value < 0.05:
        print(f"p_value = {p_value:.4f}\tГипотеза о равномерном распределении отвергается")
    else:
        print(f"p_value = {p_value:.4f}\tГипотеза о равномерном распределении не отвергается")

p_value = 0.7637	Гипотеза о равномерном распределении не отвергается
p_value = 0.0983	Гипотеза о равномерном распределении не отвергается
p_value = 0.1626	Гипотеза о равномерном распределении не отвергается
p_value = 0.4466	Гипотеза о равномерном распределении не отвергается
p_value = 0.6413	Гипотеза о равномерном распределении не отвергается
p_value = 0.6475	Гипотеза о равномерном распределении не отвергается
p_value = 0.8255	Гипотеза о равномерном распределении не отвергается
p_value = 0.8935	Гипотеза о равномерном распределении не отвергается
p_value = 0.5585	Гипотеза о равномерном распределении не отвергается
p_value = 0.2393	Гипотеза о равномерном распределении не отвергается
p_value = 0.3736	Гипотеза о равномерном распределении не отвергается
p_value = 0.6288	Гипотеза о равномерном распределении не отвергается
p_value = 0.3994	Гипотеза о равномерном распределении не отвергается
p_value = 0.6931	Гипотеза о равномерном распределении не отвергается
p_value = 0.4983	Гипотеза о равном

In [ ]:
import numpy as np
import time
import matplotlib.pyplot as plt


times = {}
for num in np.logspace(3, 7, 5, dtype=np.int32):
    start_time = time.time()
    for i in range(num):
        mt.random_int()
    # sample = [mt.random_int() for _ in range(num)]
    end_time = time.time()
    times[num] = end_time - start_time

times

{1000: 0.02030158042907715,
 10000: 0.21507859230041504,
 100000: 1.9714655876159668,
 1000000: 15.737054586410522,
 10000000: 124.50293397903442}

In [ ]:
times_stdlib = {}
for num in np.logspace(3, 7, 5, dtype=np.int32):
    start_time = time.time()
    for i in range(num):
        random.randint(0, 2**32)
    # sample = [random.randint(0, 5000) for _ in range(num)]
    end_time = time.time()
    times_stdlib[num] = end_time - start_time

times_stdlib

{1000: 0.0011317729949951172,
 10000: 0.013479232788085938,
 100000: 0.11182618141174316,
 1000000: 1.114811658859253,
 10000000: 12.42997694015503}